In [33]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress
from pathlib import Path
import tomllib

In [34]:
def get_resistance(voltage: np.ndarray, current: np.ndarray) -> np.ndarray:
    return voltage / current

## Boltzmann-data

In [35]:
BOLTZMANN_DIR = Path(__name__).parent

In [36]:
data_boltzmann = np.loadtxt(
    BOLTZMANN_DIR / "results.csv", delimiter=",", skiprows=1, unpack=True
)
voltage_data = data_boltzmann[4] / 1000

### $R_0$

In [37]:
def get_wheatstone_resistance(R1: float, R2: float, R3: float) -> float:
    return R3*R2/R1

In [38]:
with open(BOLTZMANN_DIR / "wheatstone.toml", "rb") as f:
    wheatstone_data = tomllib.load(f)

R0 = get_wheatstone_resistance(
    wheatstone_data["R1"],
    wheatstone_data["R2"] - wheatstone_data["Kabel"],
    wheatstone_data["R3"],
)

### Motstand i pæren

In [39]:
boltzmann_resistance = np.array(
    [
        get_resistance(data_boltzmann[0], data_boltzmann[2]),  # Multimeter
        get_resistance(data_boltzmann[1], data_boltzmann[3]),  # Strømforsyning
    ]
)
relative_resistance = boltzmann_resistance / R0
np.savetxt(
    BOLTZMANN_DIR / "relative_resistance.csv",
    relative_resistance,
    fmt="%.2f",
    delimiter=",",
)

### Temperatur

In [40]:
def interpolate_temperature(
    resistance: float,
    resistance_interval: tuple[float, float],
    temperature_interval: tuple[float, float],
) -> float:
    delta_R = resistance_interval[1] - resistance_interval[0]
    delta_T = temperature_interval[1] - temperature_interval[0]
    return temperature_interval[0] + delta_T / delta_R * (
        resistance - resistance_interval[0]
    )


def get_resistance_interval(
    resistance: float, resistance_array: np.ndarray
) -> tuple[int, int]:
    for index, i in enumerate(resistance_array):
        if i >= resistance:
            return index, index + 1
    return 0, 1

In [41]:
def get_temperature(
    R_rel: np.ndarray, reference_R: np.ndarray, reference_T: np.ndarray
) -> np.ndarray:
    temp_results = np.zeros_like(R_rel)
    for index, resistance in enumerate(R_rel):
        lower, upper = get_resistance_interval(resistance, reference_R)
        temp_results[index] = interpolate_temperature(
            resistance,
            (reference_R[lower], reference_R[upper]),
            (reference_T[lower], reference_T[upper]),
        )
    return temp_results

In [42]:
reference_resistance, reference_temperature, _ = np.loadtxt(
    BOLTZMANN_DIR / "resistance_temperature_table.csv", delimiter=",", skiprows=1, unpack=True
)

### Multimeter

In [49]:
temperature_results = get_temperature(relative_resistance[0], reference_resistance, reference_temperature)

res = linregress(np.log(temperature_results), np.log(voltage_data))
print(res.slope)
print(res.stderr)

4.404359187621584
0.029356787478218252


### Strømforsyning

In [48]:
temperature_results = get_temperature(relative_resistance[1], reference_resistance, reference_temperature)

res = linregress(np.log(temperature_results), np.log(voltage_data))
print(res.slope)
print(res.stderr)

4.6289023383188335
0.03237785979632769
